<a href="https://colab.research.google.com/github/ilikot/optim/blob/master/Portfolio_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Optimization

Made by Ilya Takidze, MIPT

Goal of this project is to study H.Markowitz portfolio optimization theory and to research whether some improvements or algorithms may be found. First point will be made by researching the original article and some additional ones. The second part will include working out some own thoughts and ideas.

There are two methods, which are most commonly used in portfolio optimization: Classical (Markowitz) and Tobin's models. They state different optimization problems with simmilar goal: to achieve the biggest risk-adjusted return using a given set of allowed portfolios. 

### Markowitz problem

It solves the optimization problem:
\begin{array}{r l}
maximize & \mu^T w - \gamma w^T\Sigma w\\
s.t. & {\bf{1}}^T w = 1, \quad w \in {\cal{W}}
\end{array}

where  $$r \in \mathbb{R}^n$$  is a multivariate random variable and its mean $$\mathbb{E} = \mu_r$$
and covariance matrix
$$\mathbb{E}(r − \mu_r)(r − \mu_r)^T = \Sigma_r$$
Having an initial holding of $w^0_j$ dollars of asset $j, j = 1, \dots , n$ and we invest $x_j$ dollars in asset $j$. The return of our investment is also a
random variable
$$y = r^T (w^0 + x)$$
with mean value (or expected return)
$$\mathbb{E}y = \mu^T(w^0 + x)$$
and variance (or risk)
$$(y −\mathbb{E} y)^2 = (w^0 + x)^T \Sigma_r(w^0 + x)$$

In [0]:
#!pip install chart_studio
#!pip install zipline
#!pip install pandas
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd
from scipy.optimize import minimize
import cufflinks
import chart_studio.plotly as py  
import plotly.tools as tls   
from plotly.graph_objs import *
#from zipline.utils.factory import load_bars_from_yahoo

In [0]:
def yahoo_data(stocks):
  #df = web.DataReader('TSLA', data_source='yahoo', start='2013-01-01', end='2018-01-01')
  #dates = df['']
  #a = df['Adj Close']
  num = len(stocks)
  data_source = 'yahoo'
  start = '2013-01-01'
  end = '2018-01-01'
  #data = np.zeros((num,len(a)))
  #for i in range(num):
  #  df = web.DataReader(stocks[i], data_source=data_source, start=start, end=end)
  #  data[i] = df['Adj Close']
  for symbol in stocks:
        data[symbol] = web.DataReader(symbol, data_source=data_source, start=start, end=end)['Adj Close']

  return data

In [74]:
stocks = ['GOOGL', 'TM', 'KO', 'PEP']
yahoo_data(stocks)

NameError: ignored

In [0]:
def mean_sd(returns):
    #Returns the mean and standard deviation of returns for my portfolio
    
    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # This recursion reduces outliers to keep plots pretty
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma

In [0]:
end = pd.Timestamp.utcnow()
start = end - 2500 * pd.tseries.offsets.BDay()

data = zipline.load_bars_from_yahoo(stocks=['IBM', 'GLD', 'XOM', 'AAPL', 
                                    'MSFT', 'TLT', 'SHY'],
                            start=start, end=end)

In [7]:
!pip install pandas-datareader

In [43]:
#Import Pandas web data access library

import pandas_datareader as web

#Request quote information for Tesla's stock denoted by the symbol 'TSLA'

df = web.DataReader('TSLA', data_source='yahoo', start='2010-01-01',
                           end='2014-12-31')
print(df)
stocks = ['GOOGL', 'TM', 'KO', 'PEP']
num = len(stocks)
data_source = 'yahoo'
start = '2010-01-01'
end = '2018-01-01'




                  High         Low        Open       Close    Volume  \
Date                                                                   
2010-06-29   25.000000   17.540001   19.000000   23.889999  18766300   
2010-06-30   30.420000   23.299999   25.790001   23.830000  17187100   
2010-07-01   25.920000   20.270000   25.000000   21.959999   8218800   
2010-07-02   23.100000   18.709999   23.000000   19.200001   5139800   
2010-07-06   20.000000   15.830000   20.000000   16.110001   6866900   
2010-07-07   16.629999   14.980000   16.400000   15.800000   6921700   
2010-07-08   17.520000   15.570000   16.139999   17.459999   7711400   
2010-07-09   17.900000   16.549999   17.580000   17.400000   4050600   
2010-07-12   18.070000   17.000000   17.950001   17.049999   2202500   
2010-07-13   18.639999   16.900000   17.389999   18.139999   2680100   
2010-07-14   20.150000   17.760000   17.940001   19.840000   4195200   
2010-07-15   21.500000   19.000000   19.940001   19.889999   373

In [0]:
"D- матрица доходности(обычно загружается из файла)"
D=np.array([[9.889, 11.603,11.612,  12.721,11.453,12.102],
[12.517, 13.25,12.947,12.596,12.853,13.036],
[12.786, 12.822,15.447,14.452,15.143,16.247],
[11.863, 12.114,13.359,13.437,11.913,15.300],
[11.444, 13.292,13.703,11.504,13.406,15.255],
[14.696, 15.946,16.829,17.698,16.051,17.140]],np.float64)
d= np.zeros([6,1])# столбец для средней доходности 
m,n= D.shape#размерность матрицы
for j in np.arange(0,n):
         for i in np.arange(0,m):
                  d[j,0]=d[j,0]+D[i,j]
d=d/n
print("Средняя доходность по столбцам : \n %s"%d)
CV= np.zeros([m,n])
for i in np.arange(0,m):
         for j in np.arange(0,n):
                  x=np.array(D[0:m,j]).T
                  y=np.array(D[0:m,i]).T
                  X = np.vstack((x,y))
                  CV[i,j]=round(np.cov(x,y,ddof=0)[1,0],3)
print("Ковариационная матрица  CV: \n %s"%CV)
x1,x2,x3,x4,x5,x6,x7,x8,p,q,w=symbols(' x1 x2 x3 x4 x5 x6 x7 x8  p q w' , float= True)
v1=Matrix([x1,x2,x3,x4,x5,x6])
v2=v1.T
w=0
for i in np.arange(0,m):
         for j in np.arange(0,n):
                  w=w+v1[p.subs({p:i}),q.subs({q:0})]*v2[p.subs({p:0}),q.subs({q:j})]*CV[p.subs({p:i}),q.subs({q:j})]
print("Дисперсия доходности портфеля (функция риска):\n%s"%w)                  
def objective(x):#функция риска
         x1=x[0];x2=x[1];x3=x[2]; x4=x[3]; x5=x[4];  x6=x[5]
         return 2.117*x1**2 + 3.546*x1*x2 + 4.512*x1*x3 + 4.694*x1*x4 + 4.154*x1*x5 + 3.95*x1*x6\
                + 1.903*x2**2 + 3.882*x2*x3 + 4.098*x2*x4 + 3.776*x2*x5 + 3.202*x2*x6 + 2.901*x3**2 \
                + 5.574*x3*x4 + 5.402*x3*x5 + 5.522*x3*x6 + 3.935*x4**2 + 4.928*x4*x5 + 4.63*x4*x6 \
                + 2.723*x5**2 + 4.728*x5*x6 + 3.067*x6**2
def constraint1(x):#условие для суммы долей -1
         return x[0]+x[1]+x[2]+x[3]+x[4]+x[5]-1.0
def constraint2(x): # задание доходности       
         return d[0,0]*x[0] + d[1,0]*x[1] + d[2,0]*x[2] + d[3,0]*x[3] + d[4,0]*x[4]+ d[5,0]*x[5] - 13.25
x0=[1,1,0,0,0,1]#начальное значение переменных для поиска минимума функции риска
b=(0.0,1.0)# условие для  x от нуля до единицы включая пределы
bnds=(b,b,b,b,b,b)#передача условий в функцию  риска(подготовка)
con1={'type':'ineq','fun’: constraint1} #передача условий в функцию  риска(подготовка)
con2={'type':'eq','fun’: constraint2} #передача условий в функцию  риска(подготовка)
cons=[con1,con2]#передача условий в функцию  риска(подготовка)
sol=minimize(objective,x0,method='SLSQP',\
             bounds=bnds,constraints=cons)# поиск минимума функции риска
print("Минимум функции риска -%s"%str(round(sol.fun,3)))
print("Акция 1 доля- %s, доходность- %s"%(round(sol.x[0],3),round(d[0,0]*sol.x[0],3)))
print("Акция 2 доля- %s, доходность- %s"%(round(sol.x[1],3),round(d[1,0]*sol.x[1],3)))
print("Акция 3 доля- %s, доходность- %s"%(round(sol.x[2],3),round(d[2,0]*sol.x[2],3)))
print("Акция 4 доля- %s, доходность- %s"%(round(sol.x[3],3),round(d[3,0]*sol.x[3],3)))
print("Акция 5 доля- %s, доходность- %s"%(round(sol.x[4],3),round(d[4,0]*sol.x[4],3)))
print("Акция 6 доля- %s, доходность- %s"%(round(sol.x[5],3),round(d[5,0]*sol.x[5],3)))

SyntaxError: ignored

In [0]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-04-29,203.059998,215.309998,199.110001,211.750000,209.014221,164250900
1,2019-05-06,204.289993,208.839996,192.770004,197.179993,194.632446,173663600
2,2019-05-13,187.710007,192.470001,182.850006,189.000000,187.276581,186415500
3,2019-05-20,183.520004,188.000000,177.809998,178.970001,177.338043,156970100
4,2019-05-27,178.919998,180.589996,174.990005,175.070007,173.473602,104691400
5,2019-06-03,175.600006,191.919998,170.270004,190.149994,188.416077,154348200
6,2019-06-10,191.809998,196.789993,190.300003,192.740005,190.982468,111811700
7,2019-06-17,192.899994,200.850006,192.169998,198.779999,196.967377,131658900
8,2019-06-24,198.539993,201.570007,195.289993,197.919998,196.115219,117368500
9,2019-07-01,203.169998,205.080002,200.649994,204.229996,202.367676,72879400


In [0]:
df = pd.read_csv("AAPL.csv")

### Tobin problem

For minimal risk and maximal efficiency optimization this approaches solve the following optimization problems:

$$
\begin{array}{c|c}
\begin{array}{r l}
minimize & \sqrt{\sum \limits_{i=0}^{n} w_i^2  \sigma_i^2 + 2 \sum \limits_{i=1}^{n-1} \sum \limits_{j = i+1}^n w_i  w_j  k_{ij}  \sigma_i  \sigma_j} \\
s.t. & w_0  r_0 + \sum \limits_{i=0}^{n} w_i  r_i > r_p \\
& w_0 + \sum \limits_{i=0}^{n} w_i = 1 \\
& w_i \ge 0
\end{array} 
&
\begin{array}{r l}
maximize & w_0  r_0 + \sum \limits_{i=0}^{n} w_i  r_i \\
s.t. & \sqrt{\sum \limits_{i=0}^{n} w_i^2  \sigma_i^2 + 2 \sum \limits_{i=1}^{n-1} \sum \limits_{j = i+1}^n w_i  w_j  k_{ij}  \sigma_i  \sigma_j} < \sigma_p \\
& w_0 + \sum \limits_{i=0}^{n} w_i = 1 \\
& w_i \ge 0
\end{array}
\end{array}
$$
